       This is a demo accompanying the first part of the blog post 

<html><h1 style="background: -webkit-gradient(linear,left top,left bottom,from(#fff),to(#efefef)); border-radius: 10px; box-shadow: 0 0 4px;
width: 100%; border: 1px solid grey; padding: 1em; box-sizing: border-box; margin-top: 5px; margin-bottom: 5px; text-align: center; background: brown; color: white; letter-spacing: 2px; font-weight: bold;line-height: 48px;">Integer triangles computed with Julia<br><a href="http://luschny.de/math/seq/JuliaTrianglesPart1.html">(blog)</a></h1>

Load the package Nemo from Github. ( https://github.com/Nemocas/Nemo.jl )

In [1]:
using Nemo


Welcome to Nemo version 0.16.1

Nemo comes with absolutely no warranty whatsoever



Definition: A *integer triangle* is an array of arrays whose members are integers.

An integer is a multiple precision integer which is created by the constructor ZZ.

Two examples for the creation of an integer triangle:

In [2]:
[[ZZ(k) for k in 0:n] for n in 0:6]

7-element Array{Array{fmpz,1},1}:
 [0]                  
 [0, 1]               
 [0, 1, 2]            
 [0, 1, 2, 3]         
 [0, 1, 2, 3, 4]      
 [0, 1, 2, 3, 4, 5]   
 [0, 1, 2, 3, 4, 5, 6]

The shape of the triangle is /not/ fixed. It includes the cases denoted by the OEIS keywords *tabl* and *tabf*. We allow the empty sequence to be element of an integer triangle.

In [3]:
const IntegerSequence = Array{fmpz,1}
const IntegerTriangle = Array{Array{fmpz,1},1}

Array{Array{fmpz,1},1}

In [4]:
ZZSequence(len) = Vector{fmpz}(undef, len)
ZZTriangle(dim, undef) = Vector{IntegerSequence}(undef, dim)

ZZTriangle (generic function with 1 method)

In [5]:
ZZTriangle(dim) = ZZSequence.(1:dim)
ZZTriangle(dim, f::Function) = f.(0:dim-1)

ZZTriangle (generic function with 3 methods)

In [6]:
function PrimeDivisors(n) 
    n < 2 && return fmpz[]
    sort!([p for (p, e) ∈ factor(ZZ(n))])
end

PrimeDivisors (generic function with 1 method)

In [7]:
ZZTriangle(9, PrimeDivisors)

9-element Array{Array{fmpz,1},1}:
 []    
 []    
 [2]   
 [3]   
 [2]   
 [5]   
 [2, 3]
 [7]   
 [2]   

In [8]:
import Base.sum
sum(T::IntegerTriangle) = sum.(T)

sum (generic function with 15 methods)

In [9]:
sum(PrimeDivisors.(0:7))

8-element Array{fmpz,1}:
 0
 0
 2
 3
 2
 5
 5
 7

In [10]:
S = ZZSequence(5) # isa(S, IntegerSequence)

5-element Array{fmpz,1}:
 #undef
 #undef
 #undef
 #undef
 #undef

In [11]:
T = ZZTriangle(5, undef) # isa(T, IntegerTriangle)

5-element Array{Array{fmpz,1},1}:
 #undef
 #undef
 #undef
 #undef
 #undef

In [12]:
T = ZZTriangle(6) # isa(T, IntegerTriangle)

6-element Array{Array{fmpz,1},1}:
 [#undef]                                        
 [#undef, #undef]                                
 [#undef, #undef, #undef]                        
 [#undef, #undef, #undef, #undef]                
 [#undef, #undef, #undef, #undef, #undef]        
 [#undef, #undef, #undef, #undef, #undef, #undef]

In [13]:
for n in 1:6 T[n] = [ZZ(k) for k in 1:n] end
T

6-element Array{Array{fmpz,1},1}:
 [1]               
 [1, 2]            
 [1, 2, 3]         
 [1, 2, 3, 4]      
 [1, 2, 3, 4, 5]   
 [1, 2, 3, 4, 5, 6]

In [14]:
T[3]

3-element Array{fmpz,1}:
 1
 2
 3

In [15]:
T[3][2]

2

In [16]:
typeof(T[3][2])

fmpz

In [17]:
isa(T, IntegerTriangle)

true

In [18]:
function LahIndexed(n, k)
    function recLah(n, k)
        k  < 0 && return ZZ(0)
        k == n && return ZZ(1)
        recLah(n-1, k-1) + recLah(n-1, k)*(n+k-1)
    end
    recLah(n, k)
end

[[LahIndexed(n, k) for k in 0:n] for n in 0:6]

7-element Array{Array{fmpz,1},1}:
 [1]                             
 [0, 1]                          
 [0, 2, 1]                       
 [0, 6, 6, 1]                    
 [0, 24, 36, 12, 1]              
 [0, 120, 240, 120, 20, 1]       
 [0, 720, 1800, 1200, 300, 30, 1]

Thus LahNumbers(n) returns an n-element Array{Array{BigInt,1},1}
which is according to our definiton a triangle with *n* rows.

In [19]:
const CacheLah = Dict{Int, Array{fmpz,1}}([0 => [fmpz(1)]])

function LahNumbers(n)
    haskey(CacheLah, n) && return CacheLah[n]
    prevrow = LahNumbers(n-1)
    row = Array{fmpz, 1}(undef, n+1)
    row[1] = 0; row[n+1] = 1
    for k in 2:n
        row[k] = prevrow[k-1] + prevrow[k]*(n+k-2)    
    end
    CacheLah[n] = row
end

LahNumbers (generic function with 1 method)

In [20]:
LahNumbers(5)

6-element Array{fmpz,1}:
 0  
 120
 240
 120
 20 
 1  

In [21]:
println(CacheLah)

Dict(0 => [1],4 => [0, 24, 36, 12, 1],2 => [0, 2, 1],3 => [0, 6, 6, 1],5 => [0, 120, 240, 120, 20, 1],1 => [0, 1])


Let's check the time and space consumtion: 

In [22]:
@time LahNumbers(100);

  0.004225 seconds (10.18 k allocations: 212.766 KiB)


In [23]:
LahNumbers(n, k) = LahNumbers(n+1)[k+1]
LahNumbers(5, 3)

1200

In [24]:
methods(LahNumbers)

# 2 methods for generic function "LahNumbers":
[1] LahNumbers(n) in Main at In[19]:4
[2] LahNumbers(n, k) in Main at In[23]:1

In [25]:
function LahTriangle(size) 
    length(CacheLah) < size && LahNumbers(size)
    [CacheLah[n] for n in 0:size-1] 
end

LahTriangle (generic function with 1 method)

In [26]:
methods(LahTriangle)

# 1 method for generic function "LahTriangle":
[1] LahTriangle(size) in Main at In[25]:2

In [27]:
T = LahTriangle(7)
length(T)

7

In [28]:
# Make sure that Nemo version >= 16.1
# Otherwise
# import Base.zero
# Base.zero(::Type{fmpz}) = ZZ(0)

In [29]:
sum(PrimeDivisors.(0:7))

8-element Array{fmpz,1}:
 0
 0
 2
 3
 2
 5
 5
 7

In [30]:
evensum(A) = sum(A[1:2:end]) 
oddsum(A)  = sum(A[2:2:end])
altsum(A)  = evensum(A) - oddsum(A)
evensum(T::IntegerTriangle) = evensum.(T)
oddsum(T::IntegerTriangle)  = oddsum.(T)
altsum(T::IntegerTriangle)  = evensum(T) - oddsum(T)

altsum (generic function with 2 methods)

In [31]:
T = LahTriangle(10)
println.([sum(T), evensum(T), oddsum(T), altsum(T)]);

fmpz[1, 1, 3, 13, 73, 501, 4051, 37633, 394353, 4596553]
fmpz[1, 0, 1, 6, 37, 260, 2101, 19362, 201097, 2326536]
fmpz[0, 1, 2, 7, 36, 241, 1950, 18271, 193256, 2270017]
fmpz[1, -1, -1, -1, 1, 19, 151, 1091, 7841, 56519]


In [32]:
middle(A) = A[div(end + 1, 2)] 
middle(T::IntegerTriangle) = middle.(T)

middle(LahTriangle(10)) |> println

fmpz[1, 0, 2, 6, 36, 240, 1200, 12600, 58800, 846720]


In [33]:
central(T::IntegerTriangle) = middle.(T[1:2:end])

central(LahTriangle(16)) |> println

fmpz[1, 2, 36, 1200, 58800, 3810240, 307359360, 29682132480]


In [34]:
function diag(T)
    dim = length(T)
    U = ZZTriangle(dim)
    for n in 1:dim
        R = ZZSequence(div(n+1,2))
        for k in 0:div(n-1,2)
            R[k+1] = T[n-k][k+1]
        end
        U[n] = R
    end
    U
end

diag (generic function with 1 method)

In [35]:
T = LahTriangle(10) 
diag(T)

10-element Array{Array{fmpz,1},1}:
 [1]                        
 [0]                        
 [0, 1]                     
 [0, 2]                     
 [0, 6, 1]                  
 [0, 24, 6]                 
 [0, 120, 36, 1]            
 [0, 720, 240, 12]          
 [0, 5040, 1800, 120, 1]    
 [0, 40320, 15120, 1200, 20]

In [36]:
diagsum(T) = sum(diag(T))
diagsum(LahTriangle(9))

9-element Array{fmpz,1}:
 1   
 0   
 1   
 2   
 7   
 30  
 157 
 972 
 6961

In [37]:
leftside(A)  = A[1] 
rightside(A) = A[end] 

rightside(T::IntegerTriangle) = rightside.(T)
leftside( T::IntegerTriangle) = leftside.(T)

leftside (generic function with 2 methods)

In [38]:
function profile(T::IntegerTriangle)
    println("Triangle: ");
    for row in T println(row) end; println()
    print("Sum:      "); sum(T)       |> println 
    print("EvenSum:  "); evensum(T)   |> println 
    print("OddSum:   "); oddsum(T)    |> println 
    print("AltSum:   "); altsum(T)    |> println 
    print("DiagSum:  "); diagsum(T)   |> println 
    print("Middle:   "); middle(T)    |> println 
    print("Central:  "); central(T)   |> println 
    print("LeftSide: "); leftside(T)  |> println 
    print("RightSide:"); rightside(T) |> println 
end

profile (generic function with 1 method)

In [39]:
profile(LahTriangle(10))

Triangle: 
fmpz[1]
fmpz[0, 1]
fmpz[0, 2, 1]
fmpz[0, 6, 6, 1]
fmpz[0, 24, 36, 12, 1]
fmpz[0, 120, 240, 120, 20, 1]
fmpz[0, 720, 1800, 1200, 300, 30, 1]
fmpz[0, 5040, 15120, 12600, 4200, 630, 42, 1]
fmpz[0, 40320, 141120, 141120, 58800, 11760, 1176, 56, 1]
fmpz[0, 362880, 1451520, 1693440, 846720, 211680, 28224, 2016, 72, 1]

Sum:      fmpz[1, 1, 3, 13, 73, 501, 4051, 37633, 394353, 4596553]
EvenSum:  fmpz[1, 0, 1, 6, 37, 260, 2101, 19362, 201097, 2326536]
OddSum:   fmpz[0, 1, 2, 7, 36, 241, 1950, 18271, 193256, 2270017]
AltSum:   fmpz[1, -1, -1, -1, 1, 19, 151, 1091, 7841, 56519]
DiagSum:  fmpz[1, 0, 1, 2, 7, 30, 157, 972, 6961, 56660]
Middle:   fmpz[1, 0, 2, 6, 36, 240, 1200, 12600, 58800, 846720]
Central:  fmpz[1, 2, 36, 1200, 58800]
LeftSide: fmpz[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
RightSide:fmpz[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [40]:
function InverseTriangle(T)
    dim = length(T)
    M = zeros(BigInt, dim, dim)
    for n in 1:dim, k in 1:n 
        M[n, k] = T[n][k] 
    end
    IM = inv(M)
    [[ZZ(IM[n, k]) for k in 1:n] for n in 1:dim]
end    

InverseTriangle (generic function with 1 method)

In [41]:
IT = InverseTriangle(T)      
isa(IT, IntegerTriangle) |> println
IT

true


10-element Array{Array{fmpz,1},1}:
 [1]                                                                  
 [0, 1]                                                               
 [0, -2, 1]                                                           
 [0, 6, -6, 1]                                                        
 [0, -24, 36, -12, 1]                                                 
 [0, 120, -240, 120, -20, 1]                                          
 [0, -720, 1800, -1200, 300, -30, 1]                                  
 [0, 5040, -15120, 12600, -4200, 630, -42, 1]                         
 [0, -40320, 141120, -141120, 58800, -11760, 1176, -56, 1]            
 [0, 362880, -1451520, 1693440, -846720, 211680, -28224, 2016, -72, 1]